# 00000: Model prediction and tiling for validation
Authors: Tobias G. Mueller, Mark A. Buckner
Last modified: 4 Dec 2024
Contact: __________

**Summary**: Here, we predict on an orthomosaic using out pretrained model. 
We then split the predictions and image into smaller tiles and take a random 20% for ground truthing and model validation.


This script outputs 
- predicted nest detections in yolov5 format 
- a random 20% of tiles into a testset folder 

The data used in this script was generated in:
    `AIggregation/notebooks/01_preprocessing.ipynb`

This script is followed by `03_validation_and_optimization.ipynb`. However the test tiles created in this script will need to be annotated before proceeding.

In [7]:
#imports 
import os
import fiftyone as fo
from sahi import AutoDetectionModel
from sahi.predict import get_sliced_prediction
from PIL import Image 

In [8]:
# first check the wd is not notebooks but the main folder
print("cwd is", os.getcwd())

if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")
    print("cwd changed to", os.getcwd())


cwd is /home/tmueller/github/AIggregation


# detect nests on orthomosaic using sahi

Using our trained nest detection model we predict on our stitched orthomosaic image using SAHI (slicing aided hyper-inference)   -- `https://github.com/obss/sahi`



In [ ]:
# set paths
# -------------------------------------------------------------------------------------------------------------------- #
image_directory = "datasets/drone_ortho/ortho_clip_23april.png"     # path to image to be predicted on
model_path = "AIggregation_yolov5m/weights/best.pt"                             # path to image detection model
export_directory = "datasets/export_predictions/temp"                    # directory to export model predictions to
# -------------------------------------------------------------------------------------------------------------------- #



# Import ortho image into a fiftyone dataset 
dataset_full = fo.Dataset.from_images(
    [image_directory]
)

# specify AI detection model to use for predictions
detection_model = AutoDetectionModel.from_pretrained(
    model_type='yolov5',
    model_path=model_path, #specify path to trained model
    confidence_threshold=0.25,
    device="cuda", # "cpu" or "cuda" for GPU
)

# define function for sliced predictions from sahi
def predict_with_slicing(sample, label_field, **kwargs):
    result = get_sliced_prediction(
        sample.filepath, detection_model, verbose=0, **kwargs
    )
    sample[label_field] = fo.Detections(detections=result.to_fiftyone_detections())

# predict on image, slicing at training image size
for sample in dataset_full.iter_samples(progress=True, autosave=True):
    predict_with_slicing(sample,
                         label_field="prediction",
                         slice_height=608, 
                         slice_width=608,
                         overlap_height_ratio = .4, 
                         overlap_width_ratio=.4
    )


#launch fiftyone session to see predictions
session = fo.launch_app(dataset_full)


#export predictions
dataset_full.export(
        export_dir=export_directory,
        dataset_type=fo.types.YOLOv5Dataset,
        label_field="prediction",
        include_confidence=True
    )

# Importing image, predicting, and export took
# 7m 53s using CPU (AMD Ryzen 5 5500 3.6 GHz 6-Core Processor)
# 2m 38s using GPU (EVGA SC GAMING GeForce GTX 1060 3GB 3 GB Video Card)
# 64 gb DDR4-3200 ram

# for half resolution dataset it took 38.1 seconds using GPU


 100% |█████████████████████| 1/1 [22.3ms elapsed, 0s remaining, 51.5 samples/s] 


03/27/2025 09:59:22 - INFO - eta.core.utils -    100% |█████████████████████| 1/1 [22.3ms elapsed, 0s remaining, 51.5 samples/s] 


 100% |█████████████████████| 1/1 [36.2s elapsed, 0s remaining, 0.0 samples/s] 


03/27/2025 09:59:59 - INFO - eta.core.utils -    100% |█████████████████████| 1/1 [36.2s elapsed, 0s remaining, 0.0 samples/s] 


Directory 'datasets/export_predictions/temp_lowres' already exists; export will be merged with existing files


03/27/2025 09:59:59 - WARNING - fiftyone.core.collections -   Directory 'datasets/export_predictions/temp_lowres' already exists; export will be merged with existing files


 100% |█████████████████████| 1/1 [1.3s elapsed, 0s remaining, 0.8 samples/s] 


03/27/2025 10:00:00 - INFO - eta.core.utils -    100% |█████████████████████| 1/1 [1.3s elapsed, 0s remaining, 0.8 samples/s] 


# create a test set for validation

To assess the model performance we will compare the predicitons vs a labeled random subset.

To do this, we split the predicted upon image and its detections into many smaller tiles, then randomly select 20% of them to act as our test set. 


In [ ]:
# set parameters for tiling script
# -------------------------------------------------------------------------------------------------------------------- #
source_path = export_directory                              # directory where model prediction exported to
target_path = "./datasets/testset/tiled_testset"     # directory to save tiled testset
img_ext = ".png"                                            # type of image predicted on
tile_size = 608                                             # size of tiles in pixels. default 608
test_ratio = 0.2                                            # proportion of tiles to keep for testset
overwrite = "FALSE"                                         # set to TRUE to create new testlist. default to pulling from existing list in testset folder
# -------------------------------------------------------------------------------------------------------------------- #


# set max image pixels to none
# otherwise pillow thinks large images might be a bomb DOS attack 
Image.MAX_IMAGE_PIXELS = None


# run riling script using above parameters
%run scripts/yolo_tile_modified.py -source {source_path} -target {target_path} -ext {img_ext}  -size {tile_size} -ratio {test_ratio} -overwrite {overwrite}



Image: datasets/export_predictions/temp_lowres/images/val/ortho_clip_23april.png
[(0, <POLYGON ((11991.983 260.235, 12044.631 260.235, 12044.631 321.94, 11991.983...>, 0.982088), (0, <POLYGON ((9955.655 4763.435, 10019.071 4763.435, 10019.071 4821.155, 9955.6...>, 0.981815), (0, <POLYGON ((11395.608 1034.51, 11450.445 1034.51, 11450.445 1093.079, 11395.6...>, 0.981032), (0, <POLYGON ((12129.038 355.048, 12189.58 355.048, 12189.58 413.068, 12129.038 ...>, 0.980778), (0, <POLYGON ((5869.012 2514.065, 5923.513 2514.065, 5923.513 2571.842, 5869.012...>, 0.980654), (0, <POLYGON ((11979.383 657.156, 12036.306 657.156, 12036.306 717.648, 11979.38...>, 0.980263), (0, <POLYGON ((12934.801 673.534, 12986.486 673.534, 12986.486 728.579, 12934.80...>, 0.979934), (0, <POLYGON ((11244.5 2997.623, 11298.914 2997.623, 11298.914 3054.424, 11244.5...>, 0.979904), (0, <POLYGON ((14246.323 2100.746, 14299.832 2100.746, 14299.832 2158.338, 14246...>, 0.979783), (0, <POLYGON ((13465.301 5034.873, 13525.58 5

# Annotate test set 

Before continuing to `03_validation_and_optimization.ipynb` the tiled testset must be annotated with ground truth labels.

Import the tiled images in the testset folder into labelstudio and annotate them. These will be used in the next notebook to validate the model predictions and optimize the detections. 